In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from overcomplete.sae import BatchTopKSAE, train_sae
import pandas as pd

In [2]:
train_path = "./logs/activation.pt"
ckpt = torch.load(train_path)

fc1 = ckpt["fc1"]
fc1_activations = ckpt["fc1_activations"]
labels = ckpt["labels"]

batch_size = 1024
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

dataloader = torch.utils.data.DataLoader(TensorDataset(fc1), batch_size=batch_size, shuffle=True)


/tmp/ipykernel_2186698/3713506127.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(train_path)


In [4]:
nb_concepts = 256
top_k_individual = 12

sae = BatchTopKSAE(fc1.shape[-1], nb_concepts=nb_concepts, top_k=top_k_individual*batch_size, device=device)
optimizer = torch.optim.Adam(sae.parameters(), lr=1e-3)


def criterion(x, x_hat, pre_codes, codes, dictionary):
  loss = (x - x_hat).square().mean()

  is_dead = ((codes > 0).sum(dim=0) == 0).float().detach()
  reanim_loss = (pre_codes * is_dead[None, :]).mean()

  loss -= reanim_loss * 1e-3
  return loss

logs = train_sae(sae, dataloader, criterion, optimizer, nb_epochs=25, device=device)

Epoch[1/25], Loss: 8.9883, R2: 0.3920, L0: 12.1392, Dead Features: 0.0%, Time: 159.9574 seconds
Epoch[2/25], Loss: 1.5538, R2: 0.8947, L0: 12.1392, Dead Features: 22.3%, Time: 117.2075 seconds
Epoch[3/25], Loss: 0.6518, R2: 0.9559, L0: 12.1392, Dead Features: 27.3%, Time: 116.6201 seconds
Epoch[4/25], Loss: 0.4351, R2: 0.9706, L0: 12.1392, Dead Features: 36.7%, Time: 96.3935 seconds
Epoch[5/25], Loss: 0.3367, R2: 0.9773, L0: 12.1392, Dead Features: 46.9%, Time: 81.8927 seconds
Epoch[6/25], Loss: 0.2640, R2: 0.9822, L0: 12.1392, Dead Features: 48.8%, Time: 77.5539 seconds
Epoch[7/25], Loss: 0.2182, R2: 0.9854, L0: 12.1392, Dead Features: 53.9%, Time: 88.1875 seconds
Epoch[8/25], Loss: 0.1982, R2: 0.9867, L0: 12.1392, Dead Features: 56.2%, Time: 113.9323 seconds
Epoch[9/25], Loss: 0.1881, R2: 0.9874, L0: 12.1392, Dead Features: 61.3%, Time: 105.8040 seconds
Epoch[10/25], Loss: 0.1789, R2: 0.9881, L0: 12.1392, Dead Features: 60.9%, Time: 110.0358 seconds
Epoch[11/25], Loss: 0.1707, R2: 0.